In [1]:
import mediapipe as mp
import cv2
from dollarpy import Recognizer, Template, Point
import pickle
import tkinter as tk
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [2]:
# Load the model
model_filename = 'pinching.pkl'
with open(model_filename, 'rb') as model_file:
    templates = pickle.load(model_file)

# Initialize recognizer with loaded templates
recognizer = Recognizer(templates)
print("Model loaded successfully.")

Model loaded successfully.


In [3]:
def get_screen_resolution():
    # Use tkinter to get the screen resolution
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    return root.winfo_screenwidth(), root.winfo_screenheight()

In [4]:
# Function to send "pinching" message to the server
def pinching_status_callback(pinching_detected,pinch_coordinates=None):
    if pinching_detected:
        print("Pinching")
        if pinch_coordinates:
            print(f"{pinch_coordinates[0]},{pinch_coordinates[1]}")
    # else:
        # print("Not pinching")

In [5]:
def getPointsRealTime(pinching_detected_callback):

    cap = cv2.VideoCapture(0)  # Use camera
    
    cap.set(3, 640)  # Set width
    cap.set(4, 480)  # Set height

    mp_holistic = mp.solutions.holistic
    mp_hands = mp.solutions.hands
    drawing_utils = mp.solutions.drawing_utils

    # Get the screen resolution
    screen_width, screen_height = get_screen_resolution()

    # Set up a resizable window
    window_name = "Gesture Detection"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, 1700, 980)  # Change the values as needed (e.g., 320, 240 for a smaller window)

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Flip the frame horizontally to mirror the camera feed
            frame = cv2.flip(frame, 1)
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = holistic.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw hand landmarks
            drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

            # Check for pinch gesture
            pinching_detected = False
            pinch_coordinates = None
            if results.right_hand_landmarks or results.left_hand_landmarks:
                try:
                    hand_landmarks = results.right_hand_landmarks or results.left_hand_landmarks
                    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                    
                    # Calculate distance between thumb and index fingertip to detect pinch
                    distance = ((thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2) ** 0.5

                    if distance < 0.02:  # Adjust threshold as needed
                        pinching_detected = True
                        # Calculate pinch coordinates
                        pinch_x = int(index_tip.x * image.shape[1])
                        pinch_y = int(index_tip.y * image.shape[0])
                        pinch_coordinates = (pinch_x, pinch_y)
                        # Draw a green dot at the pinch location
                        cv2.circle(image, (pinch_x, pinch_y), 10, (0, 255, 0), -1)  # Draw green dot

                except Exception as e:
                    print(f"Error processing hand landmarks: {e}")

            # Call the callback with the current pinching status and coordinates
            pinching_detected_callback(pinching_detected,pinch_coordinates)

            # Display the frame
            cv2.imshow(window_name, image)
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

In [6]:
def main():
    # Start real-time gesture detection
    getPointsRealTime(pinching_status_callback)

if __name__ == "__main__":
    main()

Pinching
930,610
Pinching
927,623
Pinching
925,623
Pinching
923,623
Pinching
929,626
Pinching
932,628
Pinching
935,626
Pinching
939,622
Pinching
937,621
Pinching
934,617
Pinching
934,623
Pinching
933,620
Pinching
933,618
Pinching
932,618
Pinching
938,618
Pinching
931,623
Pinching
931,625
Pinching
932,622
Pinching
936,621
Pinching
931,619
Pinching
942,614
Pinching
937,614
Pinching
937,618
Pinching
940,613
Pinching
936,609
Pinching
938,613
Pinching
937,611
Pinching
935,608
Pinching
935,608
Pinching
933,611
Pinching
935,605
Pinching
936,612
Pinching
938,612
Pinching
937,610
Pinching
927,609
Pinching
915,607
Pinching
918,609
Pinching
922,606
Pinching
917,611
Pinching
914,611
Pinching
917,611
Pinching
917,606
Pinching
921,607
Pinching
914,606
Pinching
916,607
Pinching
916,609
Pinching
919,606
Pinching
922,607
Pinching
914,610
Pinching
923,605
Pinching
918,608
Pinching
920,610
Pinching
917,612
Pinching
914,616
Pinching
912,609
Pinching
916,606
Pinching
918,607
Pinching
925,604
Pinching
923,6